In [2]:
import json
from zipfile import ZipFile
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re



In [2]:
# Загрузка линков из json
# --------------------------------------------
with open('data/links_list.json', 'r', encoding='utf-8') as file:
    tmp = file.read()
    links_list = json.loads(tmp)
print(links_list[400])

https://zakupki.gov.ru/epz/complaint/card/complaint-information.html?id=2256351


In [93]:
with ZipFile('data/pages.zip') as zip_file:
    item = zip_file.filelist[402]
    with zip_file.open(item.filename) as file:
        page = file.read().decode()

In [96]:
def information_about_the_results_of_consideration_of_the_complaint(page):
    """
        Сведения о результатах рассмотрения жалобы:
            Результат рассмотрения жалобы
    """
    soup = BS(page, 'html.parser')
    # print(soup.prettify())
    # Карточка с информацией
    card = soup.find('div', string="Сведения о результатах рассмотрения жалобы")
    
    try:
        card = card.find_parent().find_parent()
    except AttributeError:
        out_01 = np.NaN 
    else:
        # Содержание жалобы
        out_01 = card.find('div', class_='common-text__title', string=re.compile("Результат рассмотрения жалобы"))
        try:
            out_01 = out_01.find_next_sibling('div', class_='common-text__value').get_text().strip()
        except AttributeError:
            out_01 = np.NaN
    
    return out_01

print(information_about_the_results_of_consideration_of_the_complaint(page))

None
nan


In [17]:
# Название карточек
# Как часто встречаются названия карточек
names = !python scripts/parser_02.py
names_list = json.loads(names[0])
df = pd.DataFrame({'src': names_list})
df.value_counts()

src                                                                   
Данные участника контрактной системы в сфере закупок, подавшего жалобу    1000
Информация о субъекте контроля                                            1000
Сведения жалобы                                                           1000
Содержание жалобы                                                         1000
Информация о контрольном органе в сфере закупок                            998
Сведения о закупке                                                         996
Сведения о передаче по подведомственности                                  985
Сведения о принятии жалобы к рассмотрению                                  958
Сведения о результатах рассмотрения жалобы                                 637
Сведения о передаче по ведомственной принадлежности                         13
Предмет жалобы                                                               2
Сведения о плане закупок                                    

In [50]:
names = !python scripts/parser_03.py "Информация о субъекте контроля"
names_list = json.loads(names[0])
df = pd.DataFrame({'src': names_list})
reating = df.value_counts()
reating.head(20)

src                                                                                                                                                                              
Наименование организации                                                                                                                                                             1179
Номер телефона                                                                                                                                                                       1179
Субъект контроля                                                                                                                                                                     1179
Место нахождения                                                                                                                                                                     1175
Почтовый адрес                                                                

In [5]:
feature_out = !python scripts/parser_04.py

In [6]:
feature = pd.read_json(feature_out[0])
feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Дата размещения жалобы                              998 non-null    object 
 1   Субъект контроля                                    1000 non-null   object 
 2   Наименование организации                            1000 non-null   object 
 3   Почтовый адрес                                      998 non-null    object 
 4   Место нахождения субъекта контроля                  998 non-null    object 
 5   Номер телефона субъекта                             1000 non-null   object 
 6   Тип                                                 1000 non-null   object 
 7   Фамилия, Имя, Отчество                              631 non-null    object 
 8   Дата постановки на учет в налоговом органе          998 non-null    object 
 9 

In [8]:
feature.iloc[:,15].value_counts()

8.483000e+17    31
8.483001e+17    12
8.482000e+17    12
8.483000e+17    10
8.483000e+17    10
                ..
8.616000e+17     1
3.283000e+17     1
3.482000e+17     1
3.583004e+17     1
8.916000e+17     1
Name: Номер извещения, Length: 605, dtype: int64